In [12]:
from numpy import array ,eye, argmax
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, LSTM,Bidirectional ,Activation

 
# 1. 데이터
input_w=4
input_string="떳다떳다비행기날아라날아라높이높이날아라우리비행기내가만든비행기날아라날아라높이높이날아라우리비행기"
char_set=sorted(set(input_string))
dict_size=len(char_set)
encoder={k:v for v,k in enumerate(char_set)}
one_hot_encoder=eye(dict_size)
decoder={v:k for v,k in enumerate(char_set)}
encoded_string= [encoder[c] for c in input_string ]
one_hot_encoded_string=[one_hot_encoder[i] for i in encoded_string]

string_width=len(input_string)

output_w=string_width-input_w
x=[];y=[]

for i in range(output_w):
  x.append( one_hot_encoded_string[i:i+input_w]  )
  y.append( one_hot_encoded_string[input_w+i]    )


x=array(x)
y=array(y)
x.shape
x = x.reshape(( output_w, input_w, dict_size))


# 2. Keras Model 
model = Sequential() 
model.add(Bidirectional(LSTM(input_w, activation = 'relu',return_sequences=True), 
               input_shape=(input_w,dict_size) 
            )
)
model.add(Bidirectional(LSTM(input_w)))

model.add(Dense(dict_size) )
model.add(Activation('softmax'))
model.summary() 


Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_4 (Bidirection (None, 4, 8)              672       
_________________________________________________________________
bidirectional_5 (Bidirection (None, 8)                 416       
_________________________________________________________________
dense_4 (Dense)              (None, 16)                144       
_________________________________________________________________
activation_1 (Activation)    (None, 16)                0         
Total params: 1,232
Trainable params: 1,232
Non-trainable params: 0
_________________________________________________________________


In [13]:
 
# 3. Training 
model.compile(optimizer='adam', loss='mse') 
model.fit(x, y, epochs=100, batch_size=1) 



Epoch 1/100
46/46 [==============================] - 0s 4ms/step - loss: 0.0586
Epoch 2/100
46/46 [==============================] - 0s 4ms/step - loss: 0.0584
Epoch 3/100
46/46 [==============================] - 0s 4ms/step - loss: 0.0583
Epoch 4/100
46/46 [==============================] - 0s 4ms/step - loss: 0.0581
Epoch 5/100
46/46 [==============================] - 0s 4ms/step - loss: 0.0580
Epoch 6/100
46/46 [==============================] - 0s 4ms/step - loss: 0.0577
Epoch 7/100
46/46 [==============================] - 0s 4ms/step - loss: 0.0574
Epoch 8/100
46/46 [==============================] - 0s 4ms/step - loss: 0.0570
Epoch 9/100
46/46 [==============================] - 0s 4ms/step - loss: 0.0565
Epoch 10/100
46/46 [==============================] - 0s 4ms/step - loss: 0.0558
Epoch 11/100
46/46 [==============================] - 0s 4ms/step - loss: 0.0550
Epoch 12/100
46/46 [==============================] - 0s 4ms/step - loss: 0.0542
Epoch 13/100
46/46 [=================

In [17]:

# 4.Test 

def test_it(test_string,y,debug=True):
  x_input = array([encoder[c] for c in test_string ] ) 
  x_input=[one_hot_encoder[i] for i in x_input]
  x_input=array(x_input)
  x_input = x_input.reshape((1,input_w,dict_size))
  yhat = model.predict(x_input) 
  out=decoder[argmax(yhat)] 
  isOk= y==out
  if debug:
    print(f"# {test_string} →  {out}   {isOk}") 
  return isOk
  
print(f'# InputString : {input_string}')
okCount=0
for s in range(output_w):
  if test_it(input_string[s:input_w+s],input_string[input_w+s:input_w+s+1],True):
    okCount+=1
okPct=okCount/output_w * 100
print(f' {okPct}% : {okCount} / {output_w}')

# InputString : 떳다떳다비행기날아라날아라높이높이날아라우리비행기내가만든비행기날아라날아라높이높이날아라우리비행기
# 떳다떳다 →  비   True
# 다떳다비 →  행   True
# 떳다비행 →  기   True
# 다비행기 →  날   True
# 비행기날 →  아   True
# 행기날아 →  라   True
# 기날아라 →  날   True
# 날아라날 →  아   True
# 아라날아 →  라   True
# 라날아라 →  높   True
# 날아라높 →  이   True
# 아라높이 →  높   True
# 라높이높 →  이   True
# 높이높이 →  날   True
# 이높이날 →  아   True
# 높이날아 →  라   True
# 이날아라 →  우   True
# 날아라우 →  리   True
# 아라우리 →  비   True
# 라우리비 →  행   True
# 우리비행 →  기   True
# 리비행기 →  비   False
# 비행기내 →  리   False
# 행기내가 →  만   True
# 기내가만 →  든   True
# 내가만든 →  비   True
# 가만든비 →  행   True
# 만든비행 →  기   True
# 든비행기 →  날   True
# 비행기날 →  아   True
# 행기날아 →  라   True
# 기날아라 →  날   True
# 날아라날 →  아   True
# 아라날아 →  라   True
# 라날아라 →  높   True
# 날아라높 →  이   True
# 아라높이 →  높   True
# 라높이높 →  이   True
# 높이높이 →  날   True
# 이높이날 →  아   True
# 높이날아 →  라   True
# 이날아라 →  우   True
# 날아라우 →  리   True
# 아라우리 →  비   True
# 라우리비 →  행   True
# 우리비행 →  기   True
 95.65217391304348% : 44 / 46
